In [28]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame
from numpy import mean
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib
from IPython.display import display, HTML
import pandas

start=0
cfg_list=list()

idx1 = 62
idx2 = idx1+62
idx3 = idx2+62
idx4 = idx3+62
idx5 = idx4+62

def measure_rmse(actual, predicted,error=0):
    if len(actual)!=len(predicted):
        print("Length of data is not equals")
    error=sqrt(mean_squared_error(actual, predicted))
    return error

def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

def sarima_forecast(Data, config):
    order, sorder, trend = config
    pred=[]
    actual=[]
    actual1=[]

    for i in range(6,62):
        
        model1 = SARIMAX(Data[:i], order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
        model_fit1 = model1.fit(disp=False)
        yhat = model_fit1.predict(i, i)
        pred.append(yhat[0])
        actual.append(Data[i])
        
    
        model2 = SARIMAX(Data[idx1:idx1+i], order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
        model_fit2 = model2.fit(disp=False)
        yhat = model_fit2.predict(i, i)
        pred.append(yhat[0])
        actual.append(Data[idx1+i])
        actual1.append(Data[idx1+i])
        
      
        model3 = SARIMAX(Data[idx2:idx2+i], order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
        
        model_fit3 = model3.fit(disp=False)
        yhat = model_fit3.predict(i, i)
        
        pred.append(yhat[0])
        actual.append(Data[idx2+i])
 
        model4 = SARIMAX(Data[idx3:idx3+i], order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
        model_fit4 = model4.fit(disp=False)
        yhat = model_fit4.predict(i, i)
        pred.append(yhat[0])
        actual.append(Data[idx3+i])
        

        model5 = SARIMAX(Data[idx4:idx4+i], order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
        model_fit5 = model5.fit(disp=False)
        yhat = model_fit5.predict(i, i)
        pred.append(yhat[0])
        actual.append(Data[idx4+i])
    
    return measure_rmse(actual, pred)



def score_model(Data, n_test, cfg,count, debug=False):
    result = None
    error_resi=list()
    error_trains=list()
    error_tests=list()
    
    key = (cfg)

    try:
        with catch_warnings():
            filterwarnings("ignore")
            error_tests = sarima_forecast(Data, cfg)
    except:
        error_tests=100  
    return [key, error_tests]

def parallel_model(Data, n_test, cfg,count):
    score=[]
    global start
    start=start+1
    
    score=score_model(Data, n_test, cfg, len(cfg_list))

    print( '[%s/%s]> Model[%s]' % (int(start/5)+1,count,score))
        

    return score

def grid_search(data1,data2,data3,data4,data5, cfg_list, n_test,count, parallel=True):
    scores = []
    Data = []
    for i in range(62):
        Data.append(data1[i][0])
    for i in range(62):
        Data.append(data2[i][0])
    for i in range(62):
        Data.append(data3[i][0])
    for i in range(62):
        Data.append(data4[i][0])
    for i in range(62):
        Data.append(data5[i][0])
    
    Data = np.array(Data)
    print(cpu_count())
    if parallel:
        executor = Parallel(n_jobs=100, backend= 'multiprocessing' )
        tasks = (delayed(parallel_model)(Data, n_test, cfg, count) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        for cfg in cfg_list:
            scores=parallel_model(Data, n_test, cfg, count)
        
    scores = [r for r in scores if r[1] <100]
    scores.sort(key=lambda tup: tup[1])
    return scores 



def find_best_models(data1,data2,data3,data4,data5):

    # data split
    n_test = 16
    count=0
    
    scores=list()
    # # model configs
    cfg_list,count = sarima_configs()    

    # # grid searchech blog

    scores = grid_search(data1,data2,data3,data4,data5, cfg_list, n_test,count)

    print('-----------------------------------------------------------------------------')
    print('\t \t cfg \t\t   AVG')
    print('-----------------------------------------------------------------------------')
    for i in range(len(scores)):
        print(scores[i])
    print('-----------------------------------------------------------------------------')   
    return scores




In [29]:
PATH_TO_TEST_TRAIN_DATA = 'Tangni/Data/'

#### define dataset
series1 = read_csv(PATH_TO_TEST_TRAIN_DATA + '1-3.csv', header=0, usecols=[0,1], index_col=0) 
data1 = series1.values

series2 = read_csv(PATH_TO_TEST_TRAIN_DATA + '2-12.csv', header=0, usecols=[0,1], index_col=0) 
data2 = series2.values

series3 = read_csv(PATH_TO_TEST_TRAIN_DATA + '3-6.csv', header=0, usecols=[0,1], index_col=0) 
data3 = series3.values

series4 = read_csv(PATH_TO_TEST_TRAIN_DATA + '4-15.csv', header=0, usecols=[0,1], index_col=0) 
data4 = series4.values

series5 = read_csv(PATH_TO_TEST_TRAIN_DATA + '5-15.csv', header=0, usecols=[0,1], index_col=0) 
data5 = series5.values




In [30]:
def sarima_configs(seasonal=[0]):
    models = list()
    count=0
    # define config lists 
    p_params = [0,1,2,3,4,5]
    d_params = [0,1,2]
    q_params = [0,1,2,3,4,5]
    t_params = ['n','c','t','ct']
    P_params = [0,1,2,3,4,5]
    D_params = [0,1,2]
    Q_params = [0,1,2,3,4,5]
    m_params = [0,1,2,3,4,5]
    
#     p_params = [0]
#     d_params = [0]
#     q_params = [0]
#     t_params = ['n']
#     P_params = [0]
#     D_params = [0]
#     Q_params = [0]
#     m_params = [0]
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
                                    count=count+1
    
    return models, count

In [ ]:
scores=find_best_models(data1,data2,data3,data4,data5)

48
[1/279936]> Model[[[(0, 0, 0), (0, 0, 0, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (0, 0, 1, 0), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 0, 1, 1), 'n'], 100]]

[1/279936]> Model[[[(0, 0, 0), (0, 0, 2, 0), 'n'], 100]]

[1/279936]> Model[[[(0, 0, 0), (0, 0, 2, 1), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 0, 3, 0), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 0, 3, 1), 'n'], 100]]

[1/279936]> Model[[[(0, 0, 0), (0, 0, 4, 1), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 0, 4, 0), 'n'], 100]]

[1/279936]> Model[[[(0, 0, 0), (0, 0, 5, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (0, 1, 0, 0), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 1, 0, 1), 'n'], 100]]

[1/279936]> Model[[[(0, 0, 0), (0, 1, 1, 1), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 1, 2, 0), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 1, 3, 0), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 1, 2, 1), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (0, 1, 1, 0), 'n'], 100]]

[1/279936]> Model[[[(0, 0, 0), 

[1/279936]> Model[[[(0, 0, 0), (1, 1, 0, 4), 'n'], 1.129932536421218]]
[2/279936]> Model[[[(0, 0, 0), (2, 1, 1, 0), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (2, 1, 1, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (1, 2, 4, 4), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (1, 1, 0, 3), 'n'], 62479.53459377996]]
[1/279936]> Model[[[(0, 0, 0), (0, 1, 1, 2), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (2, 1, 2, 0), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (2, 1, 2, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (0, 2, 4, 5), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (0, 2, 5, 5), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (2, 1, 3, 0), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (2, 1, 3, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (1, 2, 5, 4), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (1, 0, 1, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (1, 2, 0, 2), 'n'], 1.4198399444190766]]
[2/279936]> Model[[[(0, 0, 0), (2, 1, 4, 0), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (2, 1, 4, 1), 

[1/279936]> Model[[[(0, 0, 0), (0, 0, 5, 3), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (3, 2, 0, 5), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (4, 0, 1, 0), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (4, 0, 1, 1), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 2, 3, 4), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 0, 0, 2), 'n'], 1.480509934664527]]
[1/279936]> Model[[[(0, 0, 0), (3, 0, 0, 4), 'n'], 2.595476065247408]]
[2/279936]> Model[[[(0, 0, 0), (1, 1, 2, 3), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (0, 0, 5, 2), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (4, 0, 2, 0), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (4, 0, 2, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (2, 2, 1, 2), 'n'], 100]][1/279936]> Model[[[(0, 0, 0), (3, 2, 5, 4), 'n'], 100]]

[2/279936]> Model[[[(0, 0, 0), (2, 2, 1, 3), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (1, 0, 2, 5), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (4, 0, 3, 0), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (4, 0, 3, 1), 'n'], 100]]
[2/

[1/279936]> Model[[[(0, 0, 0), (1, 2, 5, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 1, 4, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (1, 2, 5, 3), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (5, 1, 2, 0), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (5, 1, 2, 1), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (1, 0, 5, 3), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 1, 3, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 0, 2, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 0, 3, 2), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (4, 1, 1, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 2, 2, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (3, 2, 2, 3), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (5, 1, 3, 0), 'n'], 100]]
[3/279936]> Model[[[(0, 0, 0), (5, 1, 3, 1), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (4, 1, 3, 2), 'n'], 100]]
[1/279936]> Model[[[(0, 0, 0), (2, 0, 5, 2), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (2, 0, 2, 3), 'n'], 100]]
[3/279936]> Model[[[(0, 0, 0), 

[3/279936]> Model[[[(0, 0, 0), (0, 1, 1, 0), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (0, 1, 1, 1), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (0, 1, 0, 2), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (0, 0, 3, 2), 'c'], 742463.7367958721]]
[3/279936]> Model[[[(0, 0, 0), (4, 2, 5, 2), 'n'], 100]]
[3/279936]> Model[[[(0, 0, 0), (4, 2, 5, 3), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (0, 0, 2, 4), 'c'], 67722.8734060473]][4/279936]> Model[[[(0, 0, 0), (0, 1, 1, 5), 'c'], 100]]

[2/279936]> Model[[[(0, 0, 0), (0, 1, 2, 0), 'c'], 100]][4/279936]> Model[[[(0, 0, 0), (0, 1, 2, 1), 'c'], 100]]

[2/279936]> Model[[[(0, 0, 0), (4, 1, 3, 3), 'n'], 100]]
[3/279936]> Model[[[(0, 0, 0), (5, 0, 2, 2), 'n'], 100]]
[3/279936]> Model[[[(0, 0, 0), (5, 2, 1, 2), 'n'], 100]]
[3/279936]> Model[[[(0, 0, 0), (0, 1, 2, 5), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (0, 1, 3, 0), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (0, 1, 3, 1), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (5, 0, 3, 2), 'n'], 100]]
[3/2

[3/279936]> Model[[[(0, 0, 0), (0, 1, 5, 2), 'c'], 39693400.33467106]]
[4/279936]> Model[[[(0, 0, 0), (1, 0, 1, 4), 'c'], 1.0341014600313856]]
[4/279936]> Model[[[(0, 0, 0), (1, 1, 4, 5), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (1, 1, 5, 0), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (1, 1, 5, 1), 'c'], 100]]
[2/279936]> Model[[[(0, 0, 0), (5, 0, 1, 3), 'n'], 100]]
[2/279936]> Model[[[(0, 0, 0), (1, 0, 3, 2), 'c'], 574.1119440705203]]
[2/279936]> Model[[[(0, 0, 0), (0, 2, 4, 2), 'c'], 216276942.80996433]]
[2/279936]> Model[[[(0, 0, 0), (1, 1, 5, 5), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (1, 2, 0, 0), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (1, 2, 0, 1), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (1, 1, 2, 2), 'c'], 18127.972385166726]]
[3/279936]> Model[[[(0, 0, 0), (1, 2, 0, 3), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (1, 2, 0, 4), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (1, 0, 2, 2), 'c'], 3.3990285511692]]
[3/279936]> Model[[[(0, 0, 0), (1, 2, 1, 0), 'c'], 100]]
[3/

[6/279936]> Model[[[(0, 0, 0), (2, 2, 2, 1), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (2, 1, 1, 3), 'c'], 1.2624109121059177]]
[3/279936]> Model[[[(0, 0, 0), (2, 1, 1, 4), 'c'], 66758554795866.95]]
[5/279936]> Model[[[(0, 0, 0), (2, 2, 1, 2), 'c'], 1.6783100236531396]]
[3/279936]> Model[[[(0, 0, 0), (2, 2, 2, 3), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (2, 2, 3, 0), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (2, 2, 3, 1), 'c'], 100]]
[6/279936]> Model[[[(0, 0, 0), (2, 2, 0, 5), 'c'], 100]]
[6/279936]> Model[[[(0, 0, 0), (2, 2, 3, 3), 'c'], 100]]
[5/279936]> Model[[[(0, 0, 0), (2, 1, 3, 2), 'c'], 1838968.674500349]]
[4/279936]> Model[[[(0, 0, 0), (1, 2, 5, 2), 'c'], 1365750.7303500567]]
[4/279936]> Model[[[(0, 0, 0), (2, 2, 4, 0), 'c'], 100]]
[5/279936]> Model[[[(0, 0, 0), (2, 2, 4, 1), 'c'], 100]]
[5/279936]> Model[[[(0, 0, 0), (2, 2, 0, 2), 'c'], 1.482155620788484]]
[5/279936]> Model[[[(0, 0, 0), (2, 2, 4, 3), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (2, 1, 2, 3), 'c'], 1040.53

[4/279936]> Model[[[(0, 0, 0), (3, 2, 5, 3), 'c'], 100]]
[3/279936]> Model[[[(0, 0, 0), (3, 2, 4, 4), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (4, 0, 0, 0), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (4, 0, 0, 1), 'c'], 100]]
[6/279936]> Model[[[(0, 0, 0), (3, 2, 1, 2), 'c'], 1.7671197152270486]]
[6/279936]> Model[[[(0, 0, 0), (3, 2, 3, 5), 'c'], 100]]
[7/279936]> Model[[[(0, 0, 0), (4, 0, 0, 4), 'c'], 3.1934779149260595]]
[5/279936]> Model[[[(0, 0, 0), (3, 2, 5, 4), 'c'], 100]]
[5/279936]> Model[[[(0, 0, 0), (4, 0, 1, 0), 'c'], 100]]
[5/279936]> Model[[[(0, 0, 0), (4, 0, 1, 1), 'c'], 100]]
[9/279936]> Model[[[(0, 0, 0), (3, 2, 4, 5), 'c'], 100]]
[8/279936]> Model[[[(0, 0, 0), (3, 2, 4, 2), 'c'], 25691590.852647763]]
[4/279936]> Model[[[(0, 0, 0), (4, 0, 0, 2), 'c'], 1.7220459489245088]]
[4/279936]> Model[[[(0, 0, 0), (3, 2, 5, 5), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (4, 0, 2, 0), 'c'], 100]]
[4/279936]> Model[[[(0, 0, 0), (4, 0, 2, 1), 'c'], 100]]
[6/279936]> Model[[[(0, 0, 0